# 라이브러리 호출

In [ ]:
import pandas as pd
import numpy as np
import datetime
import glob
import os
import datetime
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from tqdm import tqdm
import plotly.express as px # express 통해서 그리기
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import networkx as nx
did_info=['현관','복도스팟1','복도스팟2','주방다운','주방','복도스팟3','거실스팟1','거실스팟2','거실1','거실2','거실3','안방','발코니1','방1','방2','욕실1','드레스룸','대피공간','옷장 센서','욕실 다운','욕실2','발코니2','식탁']

# report data, 위치정보 불러오기

In [ ]:
#데이터 불러오기
data=pd.read_csv('D:/주거_조명/1210동 1602호/report_data12_0004.csv')
data=data.drop('Unnamed: 0',axis=1)
data['hist_dt']=pd.to_datetime(data['hist_dt'])

#lid 정보 불러오기
lid=pd.read_csv('D:/주거_조명/1210동 1602호/어울림하트 아파트 1210동 1602호.csv ',encoding='cp949')

#lightNo대로 정렬
lid=lid.sort_values(by=['lightNo'])
lid=lid.reset_index(drop=True)
#조명모듈 lid만 가져오기
lid=lid.iloc[0:23]
#x좌표 y좌표 따기
x=lid['x']
y=lid['z']

# 중복된 시간 삭제하여 각 did별 데이터 프레임 만들기

In [ ]:
#중복된 시간도 삭제
for i in light_0004_did:
    globals()['data_'+str(i)]=data[data['did']==i]
    #globals()['data_'+str(i)]=globals()['data_'+str(i)].set_index('hist_dt')
    globals()['data_'+str(i)]=globals()['data_'+str(i)].rename(columns={'message':'message_'+str(i)})
    globals()['data_'+str(i)]=globals()['data_'+str(i)][['hist_dt','message_'+str(i)]]
    globals()['data_'+str(i)]=globals()['data_'+str(i)].reset_index(drop=True)
    globals()['data_'+str(i)]=globals()['data_'+str(i)][~globals()['data_'+str(i)].duplicated(subset='hist_dt')]

# P(A): 조명이 켜질 확률 구하기

In [ ]:
#각 조명 켜질 확률 구하기
for i in tqdm(light_0004_did):
    globals()['data_'+str(i)+'_on']=(globals()['data_'+str(i)])[(globals()['data_'+str(i)])['message_'+str(i)]=='0 (켜진 상태)']
    globals()['data_'+str(i)+'_on']=globals()['data_'+str(i)+'_on'].reset_index(drop=True)
for i in tqdm(light_0004_did):
    globals()['data'+str(i)+'_on_per']=len(globals()['data_'+str(i)+'_on'])/len(globals()['data_'+str(i)])
a=[]
for i in tqdm(light_0004_did):
    a.append(globals()['data'+str(i)+'_on_per'])
on={'켜질 확률':a}
result_on_per=pd.DataFrame(on,index=light_0004_did)
result_on_per

# P(A,B): 조명A,B가 동시에 켜질 확률 구하기

In [ ]:
#조명 당 같이 켜지는 확률 구하기

#조명끼리 쌍으로 데이터프레임 집합 만들기
#data_1_2->1번 디바이스 데이터프레임과 2번 디바이스 데이터프레임
for i in tqdm(light_0004_did):
    for j in light_0004_did:
        if (i!=j):
            globals()['data_'+str(i)+'_'+str(j)]=pd.merge(globals()['data_'+str(i)],globals()['data_'+str(j)])
            
#두 쌍이 동시에 켜진 데이터프레임 만들기
#data_1_2_on
for i in tqdm(light_0004_did):
    for j in light_0004_did:
        if (i!=j):
            globals()['data_'+str(i)+'_'+str(j)+'_on']=(globals()['data_'+str(i)+'_'+str(j)])[(globals()['data_'+str(i)+'_'+str(j)]['message_'+str(i)]=='0 (켜진 상태)')&(globals()['data_'+str(i)+'_'+str(j)]['message_'+str(j)]=='0 (켜진 상태)')]
            globals()['data_'+str(i)+'_'+str(j)+'_on']=globals()['data_'+str(i)+'_'+str(j)+'_on'].reset_index(drop=True)
            
            
#교집합확률 구하기
#data_1_2_on_per 1번2번 동시에 켜진 데이터프레임
#result_on_both_per 동시에 켜진 모든결과
b=[]
for i in tqdm(light_0004_did):
    for j in light_0004_did:
        if (i!=j):
            globals()['data_'+str(i)+'_'+str(j)+'_on_per']=len(globals()['data_'+str(i)+'_'+str(j)+'_on'])/len(globals()['data_'+str(i)+'_'+str(j)])
            b.append(globals()['data_'+str(i)+'_'+str(j)+'_on_per'])
on_both={'동시에 켜질 확률':b}
device_both=[]
for i in light_0004_did:
    for j in light_0004_did:
        if(i!=j):
            device_both.append((i,j))
result_on_both_per=pd.DataFrame(on_both,index=device_both)


# 신뢰도

In [ ]:
#신뢰도
for i in tqdm(light_0004_did):
    for j in light_0004_did:
        if i!=j:
            if globals()['data'+str(i)+'_on_per']==0:
                globals()['confidence_'+str(i)+'_'+str(j)]=0
            else:
                globals()['confidence_'+str(i)+'_'+str(j)]=globals()['data_'+str(i)+'_'+str(j)+'_on_per']/globals()['data'+str(i)+'_on_per']
                
#소수점 셋째자리까지 반올림
for i in tqdm(light_0004_did):
    for j in light_0004_did:
        if i!=j:
            globals()['confidence_'+str(i)+'_'+str(j)]=round(globals()['confidence_'+str(i)+'_'+str(j)],3)

# 향상도

In [ ]:
# 향상도 값 구하기
for i in tqdm(light_0004_did):
    for j in light_0004_did:
        if (i!=j):
            if globals()['data'+str(i)+'_on_per']==0:
                globals()['lift_'+str(i)+'_'+str(j)]=0
            elif globals()['data'+str(j)+'_on_per']==0:
                globals()['lift_'+str(i)+'_'+str(j)]=0
            else:
                globals()['lift_'+str(i)+'_'+str(j)]=globals()['data_'+str(i)+'_'+str(j)+'_on_per']/(globals()['data'+str(i)+'_on_per']* globals()['data'+str(j)+'_on_per'])
                
                
# 소수점 셋째자리까지 반올림

for i in tqdm(light_0004_did):
    for j in light_0004_did:
        if (i!=j):
            globals()['lift_'+str(i)+'_'+str(j)]=round(globals()['lift_'+str(i)+'_'+str(j)],3)

# 데이터프레임

In [ ]:
#신뢰도가 70프로 이상 중에 향상도값이 1이상인 데이터프레임 만들기


#향상도 신뢰도 다 고려한것
result3_device=[]
result3_per=[]
lift_per=[]
for i in tqdm(light_0004_did):
    for j in light_0004_did:
        if (i!=j):
            if globals()['confidence_'+str(i)+'_'+str(j)]>0.7:
                if globals()['lift_'+str(i)+'_'+str(j)]>1:
                    result3_device.append((did_info[i-1],did_info[j-1]))
                    result3_per.append(globals()['confidence_'+str(i)+'_'+str(j)])
                    lift_per.append(globals()['lift_'+str(i)+'_'+str(j)])
                
test2134=['일치','일치','불일치','일치','일치','불일치','일치','일치','불일치','불일치','불일치',
        '일치','불일치','일치','일치','불일치','일치' ]
#결과값 데이터프레임
index=pd.MultiIndex.from_tuples(result3_device)
result3=pd.DataFrame({'A가 켜졌을때 B가 켜질 확률(신뢰도)':result3_per,'향상도':lift_per,'검정':test2134},index=index)
result3.index.names=["A","B"]
result3


# Networkx를 이용한 시각화

In [ ]:
#노드 모형은 분류안하고 신뢰도에 따라 엣지 색깔 및 모형 분류, 노드를 사용시간에 따라 분류


#H 그래프
H = nx.DiGraph()
#did로 노드 추가
H.add_nodes_from(did_info)
#노드속성 켜진 시간으로 
for i in [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,19,20,21,22,23]:
    H.nodes[did_info[i-1]]['on_time']=df1['time'].loc[i,'0 (켜진 상태)']
    H.nodes[did_info[i-1]]['on']=0
H.nodes[did_info[18-1]]['on_time']=0
H.nodes[did_info[18-1]]['on']=1
#node 속성에 따라 크기 설정하기 위해 node 속성만 가져옴
size_node=nx.get_node_attributes(H,'on_time')
size_device=[size_node.get(node) for node in H.nodes()]
#노드 위치 추가
#노두 위치 추가
test_lid={}
for i in range(23):
    test_lid[did_info[i]]=[x[i],y[i]]
test_lid
#색상 변경
#색상 변경
color_map=nx.get_node_attributes(H,'on_time')
for key in color_map:
    if color_map[key]==0:
        color_map[key]="tab:blue"
    elif color_map[key]>=284:
        color_map[key]='tab:orange'
    else:
        color_map[key]='tab:blue'
color_device=[color_map.get(node) for node in H.nodes()]


#노드 모형
shape_map=nx.get_node_attributes(H,'on_time')
for key in shape_map:
    if shape_map[key]==0:
        shape_map[key]="o"
    elif shape_map[key]>=284:
        shape_map[key]='s'
    else:
        shape_map[key]='^'
shape_device=[shape_map.get(node) for node in H.nodes()]





#엣지추가
for i in tqdm(light_0004_did):
    for j in light_0004_did:
        if i!=j:
            if globals()['confidence_'+str(i)+'_'+str(j)]>0.5:
                H.add_edges_from(globals()['pattern50_'+str(i)+'_'+str(j)])
#엣지 색깔
edge_color_map=nx.get_edge_attributes(H,'confidence')
for key in edge_color_map:
    if edge_color_map[key]>=0.7:
        edge_color_map[key]='green'
    elif (edge_color_map[key]>=0.5)&(edge_color_map[key]<0.6):
        edge_color_map[key]='black'
    elif (edge_color_map[key]>=0.6)&(edge_color_map[key]<0.7):
        edge_color_map[key]='purple'
edge_colors=[edge_color_map.get(edge) for edge in H.edges()]            

#엣지 정보
edge_labels = dict([((n1, n2), d['confidence'])
                    for n1, n2, d in H.edges(data=True)])
#엣지 넓이
edges = H.edges()
test=[H[u][v]['confidence'] for u,v in edges]

#엣지 스타일
edge_style_map=nx.get_edge_attributes(H,'confidence')
for key in edge_style_map:
    if edge_style_map[key]>=0.7:
        edge_style_map[key]='dashed'
    elif (edge_style_map[key]>=0.5)&(edge_style_map[key]<0.6):
        edge_style_map[key]='solid'
    elif (edge_style_map[key]>=0.6)&(edge_style_map[key]<0.7):
        edge_style_map[key]='dotted'
edge_styles=[edge_style_map.get(edge) for edge in H.edges()]    

#마무리
plt.figure(figsize=(15,9))
#plt.title('1210동 1602호 조명 패턴(아파트 실제 배치도)',fontsize=30,fontweight='heavy')
#plt.text(20,-120,'신뢰도 50%~59%: 검은색(실선),   60%~69%: 보라색(점선),   70%~100%: 초록색(파선)',fontsize=20)
#plt.text(130,-95,'노드 색깔 구분',fontsize=20)
#plt.text(130,-100,'orange: 조명 사용시간 상위 40프로',fontsize=20,color='tab:orange')
#plt.text(130,-105,'blue: 조명 사용시간 하위 60프로',fontsize=20,color='tab:blue')
#plt.text(130,-110,'red:  설치 취소 ',fontsize=20,color='tab:red')
#plt.text(130,-115,'초록색선은 신뢰도가 70프로 이상',fontsize=10)
plt.rc('font', family='Malgun Gothic')

nx.draw(H,test_lid, with_labels=True,node_color=color_device,font_color="black",font_family='Malgun Gothic',edge_color=edge_colors,font_weight='heavy',width=2.5,style=edge_styles,font_size=22)



nx.draw_networkx_edge_labels(H, test_lid, edge_labels=edge_labels, label_pos=0.5,
                             font_color='tab:red', font_size=12)

plt.savefig('D:/주거_조명/1210동 1602호/1210동_1602호_조명패턴1(논문용).png')
plt.show()

In [ ]:
#논문용으로 모형까지 분류해서 급하게 만든 노드 
#이떄는 사용시간에 따라 노드 모형 색깔이 분류되고 신뢰도에 따라 엣지 모형이랑 색깔이 구분된다


#H 그래프
H = nx.DiGraph()
#did로 노드 추가
H.add_nodes_from(did_info)
#노드속성 켜진 시간으로 
# 급하게 하느라고 숫자를 직접지정 이건 설취취소된 리스트를 미리 만들어야함
for i in [1,3,4,5,6,9,10,11,12,13,15,16,19,20,21,22,23]:
    H.nodes[did_info[i-1]]['on_time']=df1['time'].loc[i,'0 (켜진 상태)']
    H.nodes[did_info[i-1]]['on']=0
H.nodes[did_info[1]]['on_time']=0
H.nodes[did_info[1]]['on']=1
H.nodes[did_info[6]]['on_time']=0
H.nodes[did_info[6]]['on']=1
H.nodes[did_info[7]]['on_time']=0
H.nodes[did_info[7]]['on']=1
H.nodes[did_info[13]]['on_time']=1
H.nodes[did_info[13]]['on']=1
H.nodes[did_info[16]]['on_time']=0
H.nodes[did_info[16]]['on']=1
H.nodes[did_info[17]]['on_time']=1
H.nodes[did_info[17]]['on']=1
#node 속성에 따라 크기 설정하기 위해 node 속성만 가져옴
size_node=nx.get_node_attributes(H,'on_time')
size_device=[size_node.get(node) for node in H.nodes()]
#노드 위치 추가
#노두 위치 추가
pos={}
for i in range(23):    #숫자로 한 이유는 lid정보를 데이터프레임으로 가져와서 range(23)으로 함
    pos[did_info[i]]=[x[i],y[i]]
#색상 변경
#색상 변경
color_map=nx.get_node_attributes(H,'on_time')
for key in color_map:
    if color_map[key]==0:
        color_map[key]="tab:red"
    elif color_map[key]>=222:
        color_map[key]='tab:orange'
    else:
        color_map[key]='tab:blue'
color_device=[color_map.get(node) for node in H.nodes()]
#엣지추가
for i in tqdm(light_0004_did):
    for j in light_0004_did:
        if i!=j:
            if globals()['confidence_'+str(i)+'_'+str(j)]>0.5:
                H.add_edges_from(globals()['pattern50_'+str(i)+'_'+str(j)])
#엣지 색깔
edge_color_map=nx.get_edge_attributes(H,'confidence')
for key in edge_color_map:
    if edge_color_map[key]>=0.7:
        edge_color_map[key]='green'
    elif (edge_color_map[key]>=0.5)&(edge_color_map[key]<0.6):
        edge_color_map[key]='black'
    elif (edge_color_map[key]>=0.6)&(edge_color_map[key]<0.7):
        edge_color_map[key]='purple'
edge_colors=[edge_color_map.get(edge) for edge in H.edges()]            

#엣지 정보
edge_labels = dict([((n1, n2), d['confidence'])
                    for n1, n2, d in H.edges(data=True)])
#엣지 넓이
edges = H.edges()
test=[H[u][v]['confidence'] for u,v in edges]

box2 = {'boxstyle': 'square',
        'ec': (0.5, 0.5, 1.0),
        'fc': (0.8, 0.8, 1.0),
        'linestyle': '--'}

#엣지 스타일
edge_style_map=nx.get_edge_attributes(H,'confidence')
for key in edge_style_map:
    if edge_style_map[key]>=0.7:
        edge_style_map[key]='dashed'
    elif (edge_style_map[key]>=0.5)&(edge_style_map[key]<0.6):
        edge_style_map[key]='solid'
    elif (edge_style_map[key]>=0.6)&(edge_style_map[key]<0.7):
        edge_style_map[key]='dotted'
edge_styles=[edge_style_map.get(edge) for edge in H.edges()]    

#마무리
plt.figure(figsize=(15,9))
nx.draw_networkx_nodes(H, pos,nodelist=['거실2','거실3','방2','욕실1','주방','식탁','옷장 센서'],node_shape='^',node_color='tab:orange',label='스마트조명 사용시간 상위 40%')
nx.draw_networkx_nodes(H, pos,nodelist=['대피공간','발코니1','안방','욕실2','욕실 다운','발코니2','거실1','복도스팟3','주방다운','복도스팟2','방1','현관'],node_shape='o',node_color='tab:blue',label='스마트조명 사용시간 하위 60%')
nx.draw_networkx_nodes(H, pos,nodelist=['드레스룸','거실스팟2','거실스팟1','복도스팟1'],node_shape='s',node_color='tab:red')

nx.draw_networkx_labels(H, pos,  font_size=20, font_color="black",font_family='Malgun Gothic',font_weight='heavy')
nx.draw_networkx_edges(H,pos,width=2.5,edge_color=edge_colors,style=edge_styles)
nx.draw_networkx_edge_labels(H, pos, edge_labels=edge_labels, label_pos=0.5,
                             font_color='tab:red', font_size=12)
plt.axis('off')
plt.savefig('D:/주거_조명/1208동 2202호/1208동_2202호_조명패턴최종(논문용).png')
plt.show()




